In [10]:
import timeit
import importlib
import relevancer as rlv
import pandas as pd
from sklearn.naive_bayes import MultinomialNB 
from bson.objectid import ObjectId

In [11]:
my_token_pattern=r"[#@]?\w+\b|[\U00010000-\U0010ffff]"

In [12]:
collection = 'coll123' 

In [13]:
rlvdb, rlvcl = rlv.connect_mongodb(configfile='myconfig.ini',coll_name=collection)

In [14]:
importlib.reload(rlv)
begin = ObjectId('55950fb4d04475ee9867f3a4')
end = ObjectId('55950fc9d04475ee986841c3')

annotated_tw_ids = ['563657483395530753', '563662532326330370', '563654330041909248', '563654944927281152', '563657924233289728', '563661021559390208', '563651950386757632', '563657164317667328', '563660271810383872', '563662538949160960'] #You should get the actual annotated tweet ids from the annotated tweets collection.

tweetlist = rlv.read_json_tweet_fields_database(rlvcl, mongo_query=({'_id': {'$gte': begin, '$lte': end},'lang':'en'}), tweet_count=5000, annotated_ids=annotated_tw_ids)#=tweetsDF)

rlv.logging.info("Number of tweets:" + str(len(tweetlist)))
print("Number of tweets:",len(tweetlist))

Number of tweets: 4992


In [15]:
tweetsDF = rlv.create_dataframe(tweetlist)

4992


In [16]:
tok = rlv.tok_results(tweetsDF, elimrt = True)


Available attributes of the tweets: Index(['id_str', 'is_retweet', 'screen_name', 'text', 'texttok'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4992 entries, 0 to 4991
Data columns (total 5 columns):
id_str         4992 non-null object
is_retweet     4992 non-null bool
screen_name    4992 non-null object
text           4992 non-null object
texttok        4992 non-null object
dtypes: bool(1), object(4)
memory usage: 199.9+ KB

tweet set summary: None
0    Solar Powered Motion Sensor Security Wall Floo...
1    Brian Williams claimed he saw a 'body floating...
2    High Lumens CREE 36W LED Work Light Bars Flood...
3    Centre to unveil schemes for people affected b...
4    18W CREE 4"x4" LED WORK LIGHT FLOOD BEAM OFFRO...
Name: text, dtype: object

tweets are NOT tokenized.


In [40]:
importlib.reload(rlv)
tst_https = tweetsDF[tweetsDF.text.str.contains("https")]#["text"]
tst_http = tweetsDF[tweetsDF.text.str.contains("http:")]#["text"]
tstDF = tst_http

tstDF = rlv.normalize_text(tstDF)
print(tstDF["text"])#.iloc[10])
#rlv.logging.info("This text overwritten by tokenizer" + str(tstDF["text"]))
print("normalization:",tstDF["active_text"]) #.iloc[10])
#rlv.logging.info("This text overwritten by normalization" + str(tstDF["active_text"]))

0       Solar Powered Motion Sensor Security Wall Floo...
1       Brian Williams claimed he saw a 'body floating...
2       High Lumens CREE 36W LED Work Light Bars Flood...
4       18W CREE 4"x4" LED WORK LIGHT FLOOD BEAM OFFRO...
6       Cars &amp; Deals : 10x 27W 12V 24V LED Work Li...
7       #IBNA First victim of floods in Albania: Tiran...
8       4 Simple Steps to Writing a Blog Post That Flo...
11      Eco-tourism cover begetter regarding flood ope...
12      Cars Deals : 10x 27W 12V 24V LED Work Light FL...
13      I won 2007/13 poll by a landslide only to be r...
15      Kenya donates $1m to Malawi flood victims: Sen...
16      “@RealJamesWoods: I'm sorry, why was he drinki...
17      Thank You sa Flood @reinborja #NewFriend http:...
20      Cars &amp; Deals : 10x 27W 12V 24V LED Work Li...
21      THE BEST #DEALS &gt;&gt; 10x 27W 12V 24V LED W...
23      Did Brian Williams also lie during Hurricane K...
24      Kashmir Floods: World Bank, J&amp;K Govt Wrap ...
25      Cars D

/home/elif/Project/Relevancer/relevancer.py:368: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  mytextDF["active_text"] = mytextDF[tok_result_col].apply(lambda tw: re.sub(http_re, 'urlurlurl', tw))


In [18]:
importlib.reload(rlv)
find_distance = rlv.get_vectorizer_and_distance(tstDF)
#print(find_distance)
%timeit res  = find_distance

Kume sayisi 1: 410
Kume sayisi 2: 410
10000000 loops, best of 3: 24.3 ns per loop


In [19]:
setup="from relevancerdb import find_distance"
t = timeit.Timer("find_distance", setup=setup)
t.timeit()

492

Available attributes of the tweets: Index(['id_str', 'is_retweet', 'screen_name', 'text', 'texttok'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 492 entries, 0 to 491
Data columns (total 5 columns):
id_str         492 non-null object
is_retweet     492 non-null bool
screen_name    492 non-null object
text           492 non-null object
texttok        492 non-null object
dtypes: bool(1), object(4)
memory usage: 19.7+ KB

tweet set summary: None
0    Solar Powered Motion Sensor Security Wall Floo...
1    Brian Williams claimed he saw a 'body floating...
2    High Lumens CREE 36W LED Work Light Bars Flood...
3    Centre to unveil schemes for people affected b...
4    18W CREE 4"x4" LED WORK LIGHT FLOOD BEAM OFFRO...
Name: text, dtype: object

tweets are NOT tokenized.
In get_tw_ids
Length of the tweet ids and the first then ids 20000 ['563657483395530753', '563662532326330370', '563654330041909248', '563654944927281152', '563657924233289728', '563661021559390208

/home/elif/Project/Relevancer/relevancer.py:368: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  mytextDF["active_text"] = mytextDF[tok_result_col].apply(lambda tw: re.sub(http_re, 'urlurlurl', tw))


0.015858312995987944

In [43]:
importlib.reload(rlv)
print(timeit.timeit("from relevancerdb import find_distance", setup="from relevancer import get_vectorizer_and_distance"))

1.4297529140021652


In [21]:

start_tweet_size = len(tweetsDF)
rlv.logging.info("\nNumber of the tweets after retweet elimination:"+ str(start_tweet_size))

In [22]:
tw_id_list = rlv.get_ids_from_tw_collection(rlvcl)
print ("Length of the tweet ids and the first then ids",len(tw_id_list),tw_id_list[:10])

In get_tw_ids
Length of the tweet ids and the first then ids 20000 ['563657483395530753', '563662532326330370', '563654330041909248', '563654944927281152', '563657924233289728', '563661021559390208', '563651950386757632', '563657164317667328', '563660271810383872', '563662538949160960']


In [23]:
cluster_list = rlv.create_clusters(tweetsDF, my_token_pattern, nameprefix='1-')

print (len(cluster_list))  
a_cluster = cluster_list[0]

In create_clusters
15


In [28]:
print("cluster_no", a_cluster['cno'] )

print("cluster_str", a_cluster['cstr'] )

print("cluster_tweet_ids", a_cluster['twids'] )

print("cluster_freq", a_cluster['rif'] )

print("cluster_prefix", a_cluster['cnoprefix'] )

print("cluster_tuple_list", a_cluster['ctweettuplelist'] )

print("cluster_entropy", a_cluster['user_entropy'] )

cluster_no 1
cluster_str cluster number and size are: 1    358

cluster_tweet_ids ['563130097696129024', '563129721966170114', '563129460862353410', '563129729763381248', '563129728047906816', '563130118248222720', '563130290105643009', '563129452209516544', '563130089580134400', '563129463722876928', '563130105057140736', '563130118034296832', '563130324880596993', '563130125659537408', '563129469724921857', '563130106650976256', '563130309978247168', '563129727032901632', '563129741943648257', '563129459784445952', '563134704853868544', '563134677922238466', '563142333588176897', '563134311042281472', '563134688143757312', '563141376330575872', '563134688168902657', '563141353215762432', '563134697220214784', '563130087982112768', '563129857865830400', '563129298576343040', '563128360499683329', '563129840706924544', '563133613533696002', '563129852039925760', '563129281878831105', '563129301382340608', '563130081187352576', '563129292167446529', '563129296089145348', '56312983470905

In [31]:
collection_name = collection + '_clusters'
rlvdb[collection_name].insert(cluster_list) #Each iteration results with a candidate cluster list. Each iteration will have its own list. Therefore they are not mixed.
print("Clusters were written to the collection:", collection_name)

Clusters were written to the collection: coll123_clusters


In [26]:
tweets_as_text_label_df = pd.DataFrame({'label' : ['relif', 'social'] , 'text' : ["RT @OliverMathenge: Meanwhile, Kenya has donated Sh91 million to Malawi flood victims, according to the Ministry of Foreign Affairs." , "Yow ehyowgiddii! Hahaha thanks sa flood! #instalike http://t.co/mLaTESfunR"]})
print("tweets_as_text_label_df:", tweets_as_text_label_df)

tweets_as_text_label_df:     label                                               text
0   relif  RT @OliverMathenge: Meanwhile, Kenya has donat...
1  social  Yow ehyowgiddii! Hahaha thanks sa flood! #inst...


In [27]:
# get vectorizer and classifier
vect_and_classifier = rlv.get_vectorizer_and_mnb_classifier(tweets_as_text_label_df, my_token_pattern, pickle_file="vectorizer_and_classifier_dict")
vectorizer, mnb_classifier = vect_and_classifier["vectorizer"], vect_and_classifier["classifier"]

In get_mnb_classifier:
Pickle file was written to vectorizer_and_classifier_dict.pickle


In [ ]:
# get label for a new tweet:
ntw = vectorizer.transform(["Why do you guys keep flooding TL with smear campaign for a candidate you dont like.You think you can actually influnece people's decision?"])
predictions = mnb_classifier.predict(ntw)
print("Predictions:", predictions)

In [ ]:
rlv.logging.info('\nscript finished')